In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import recall_score, accuracy_score, precision_score,classification_report, confusion_matrix, roc_curve, roc_auc_score,auc
from pandas import set_option
import seaborn as sns

In [2]:
set_option('display.max_row', 10)
set_option('display.width', 100)
set_option('precision', 2)
set_option('display.max_column', 40)

In [3]:
df_korban = pd.read_csv('datafraud.csv') 
df_non_korban = pd.read_csv('datanonfraud.csv')
frames= [df_korban, df_non_korban] # SAMPEL TERDIRI DARI 10000 BARIS YANG DIAMBIL SECARA ACAK
data= pd.concat(frames)
y_true = data['LABEL']
data.head()


C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
0,0,201010170,0,1.88e+06,0,2540,ATM_BRI,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00
1,0,205010360,0,0.00e+00,1800000,2612,ATM_BRI,BRITAMA,TransferBRI,True,dalam,0,1,1,0,1.80e+06,1.80e+06,1.80e+06,0.96
2,0,201010170,0,8.29e+04,0,2629,ATM_BRI,BRITAMA,CekSaldo,False,dalam,0,0,1,89,0.00e+00,0.00e+00,0.00e+00,0.00
3,0,201010170,0,2.40e+06,0,22094,INTERNET BANKING,SIMPEDES,CekSaldo,False,dalam,0,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00
4,0,201010170,0,7.84e+06,0,25557,SMS BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00


In [4]:
data.shape

(309290, 19)

In [5]:
data.duplicated().sum()

3525

In [6]:
data.loc[data.duplicated(keep = False), :] 

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
1199,0,201010170,0,2.44e+06,0,38100,INTERNET BANKING,SIMPEDES,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
1200,0,201010170,0,2.44e+06,0,38100,INTERNET BANKING,SIMPEDES,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
9616,0,201010170,0,1.14e+07,0,66088,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
9617,0,201010170,0,1.14e+07,0,66088,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
16225,0,201010170,0,6.63e+06,0,48294,INTERNET BANKING,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272825,0,201010170,0,8.11e+05,0,73580,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
272831,0,201010171,0,6.57e+04,100000,44302,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,100000.0,0.00
272836,0,201010170,0,3.39e+06,0,49102,ATM_BRI,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.0,0.00
272837,0,103010020,0,3.91e+07,500000,50923,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.0,0.01


In [7]:
data=data.drop_duplicates(keep= 'first')

In [8]:
data.responcode =pd.to_numeric(data.responcode, errors='coerce').fillna(-1, downcast='infer') # TERDAPAT NAN YANG HARUS DIHILANGKAN
data.responcode_before =pd.to_numeric(data.responcode_before, errors='coerce').fillna(-1, downcast='infer')

In [9]:
data[data['LABEL']==1].describe()  

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1179.00,1.18e+03,1.18e+03,1179.00,1179.0,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,14.80,2.23e+07,2.44e+06,42452.84,12.0,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,25.06,1.89e+08,9.12e+06,20143.72,23.2,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00,0.00e+00,1.00e+03,8.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00,0.00e+00,4.00e+05,27799.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,0.00,7.23e+05,1.25e+06,41328.00,0.0,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,51.00,6.56e+06,2.00e+06,59873.00,0.0,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,93.00,2.87e+09,2.65e+08,85168.00,61.0,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [10]:
data= data.dropna()

In [11]:
one_hot = pd.get_dummies(data['responcode']) # RESPONCODE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [12]:
one_hot = pd.get_dummies(data['responcode_before'])# RESPONCODE_BEFORE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode_before', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [13]:
one_hot = pd.get_dummies(data['terminaltype'])
data= data.drop('terminaltype', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [14]:
one_hot = pd.get_dummies(data['jenis_kartu'])
data= data.drop('jenis_kartu', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [15]:
one_hot = pd.get_dummies(data['groupname'])
data= data.drop('groupname', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [16]:
one_hot = pd.get_dummies(data['dana_keluar'])
data= data.drop('dana_keluar', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [17]:
one_hot = pd.get_dummies(data['lokasi'])
data= data.drop('lokasi', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [18]:
# FITUR ID SUDAH DIWALIKI OLEH GROUPNAME
data.drop(['fiturid'], axis=1, inplace=True)
data.head() # fitur id sudah dibuang

,LABEL,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,-1,0,3,5,30,51,53,61,66,...,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Meps,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
0,0,1.88e+06,0,2540,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
1,0,0.00e+00,1800000,2612,1,1,0,1.80e+06,1.80e+06,1.80e+06,0.96,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
2,0,8.29e+04,0,2629,0,1,89,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
3,0,2.40e+06,0,22094,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
4,0,7.84e+06,0,25557,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0


In [19]:
X = data.iloc[:, data.columns != "LABEL"]
Y = data.iloc[:, data.columns == "LABEL"]

In [20]:
X.head()

,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,-1,0,3,5,30,51,53,61,66,68,...,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Meps,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
0,1.88e+06,0,2540,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
1,0.00e+00,1800000,2612,1,1,0,1.80e+06,1.80e+06,1.80e+06,0.96,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
2,8.29e+04,0,2629,0,1,89,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
3,2.40e+06,0,22094,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
4,7.84e+06,0,25557,0,1,0,0.00e+00,0.00e+00,0.00e+00,0.00,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0


In [21]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.37, random_state= None) # UKURAN DATA TESTING YAKNI 2000 BARIS

In [22]:
# STANDARDISASI NILAI PADA SETIAP FITUR
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [23]:
sm = SMOTE(random_state=None)
X_train_ov, y_train_ov = sm.fit_sample(X_train_std, y_train.values.reshape(-1,))

In [24]:
y_train_ov[y_train_ov==1].shape


(191875,)

In [25]:
y_train_ov[y_train_ov==0].shape


(191875,)

In [26]:
print('SETELAH DILAKUKAN OVERSAMPLING, DIMENSI DARI X_TRAIN YAKNI: {}'.format(X_train_ov.shape))
print('SETELAH DILAKUKAN OVERSAMPLING, DIMENSI DARI y_TRAIN YAKNI: {}\n'.format(y_train_ov.shape))

print("SETELAH OVERSAMPLING, JUMLAH LABEL '1': {}".format(sum(y_train_ov==1)))
print("SETELAH OVERSAMPLING, JUMLAH LABEL '0': {}".format(sum(y_train_ov==0)))

SETELAH DILAKUKAN OVERSAMPLING, DIMENSI DARI X_TRAIN YAKNI: (383750, 69)
SETELAH DILAKUKAN OVERSAMPLING, DIMENSI DARI y_TRAIN YAKNI: (383750,)

SETELAH OVERSAMPLING, JUMLAH LABEL '1': 191875
SETELAH OVERSAMPLING, JUMLAH LABEL '0': 191875


In [ ]:
import time
start = time.time()
clf= SVC(C=10.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True,
         probability=True, tol=0.001, cache_size=200, 
         class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

clf.fit(X_train_ov, y_train_ov)
pred = clf.predict(X_test_std)
print(classification_report(y_test, pred))
print('AKURASI PADA DATA TRAINING SVM DENGAN SMOTE:',clf.score(X_train_ov, y_train_ov))
print('AKURASI PADA DATA TESTING SVM DENGAN SMOTE:', clf.score(X_test_std, y_test))
matrix = confusion_matrix(y_test, pred)
sns.heatmap(matrix, cmap="coolwarm_r", annot=True, linewidths=0.5)
pyplot.title("CONFUSION MATRIX")
pyplot.xlabel("PREDIKSI")
pyplot.ylabel("REALITA")
pyplot.show()
end = time.time()
print("WAKTU EKSEKUSI PROGRAM (dalam detik) yakni:",end - start)


In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.cross_validation import StratifiedKFold
import numpy 
X_train2 = X_train_ov[:, numpy.arange(0, 43, 1)]

cv = StratifiedKFold(y_train_ov, n_folds=3 ,random_state=1)
fig = pyplot.figure(figsize=(7, 5))
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (train, test) in enumerate(cv):
    probas = clf.fit(X_train2[train],y_train_ov[train]).predict_proba(X_train2[test])
    fpr, tpr, thresholds = roc_curve(y_train_ov[test], probas[:, 1],
    pos_label=1)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    pyplot.plot(fpr,tpr,lw=1,label='ROC fold %d (area = %0.2f)'% (i+1, roc_auc))
pyplot.plot([0, 1],[0, 1],linestyle='--',color=(0.6, 0.6, 0.6),label='random guessing')
mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
pyplot.plot(mean_fpr, mean_tpr, 'k--',
label='mean ROC (area = %0.2f)' % mean_auc, lw=2)
pyplot.plot([0, 0, 1],[0, 1, 1],lw=2,linestyle=':',color='black',
label='perfect performance')
pyplot.xlim([-0.05, 1.05])
pyplot.ylim([-0.05, 1.05])
pyplot.xlabel('FALSE POSITIVE RATE')
pyplot.ylabel('TRUE POSITIVE RATE/ RECALL')
pyplot.title('Receiver Operator Characteristic')
pyplot.legend(loc="lower right")
pyplot.show()                         
                     